In [51]:
import torch
import torch.nn as nn
import tools.AR_data as ar
import numpy as np

In [86]:
data = ar.ARData(num_datapoints=500, num_prev=3, num_aft=1, noise_var=1)

In [87]:
data.y_train.shape

(400, 1)

In [88]:
data.X_train.shape

(400, 3)

In [8]:
print(data.X_train[0:3])
print("---------")
data.y_train[0:3]

[[-0.14317337  0.06001333 -2.00518375]
 [ 0.06001333 -2.00518375  0.53369902]
 [-2.00518375  0.53369902 -0.7116704 ]]
---------


array([[ 0.53369902, -0.7116704 , -0.52469723],
       [-0.7116704 , -0.52469723, -0.99014573],
       [-0.52469723, -0.99014573, -0.78697895]])

In [67]:
# data.x.shape
data.y_train.shape

(40, 1)

In [91]:
n_epochs = 100
batch_size = 1

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.lstm = nn.LSTM(1,2)
    
    def forward(self, x):
        out, hid = self.lstm(x)
        
        return out
    
model = Model()
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

X = data.X_train
y = data.y_train

for ep in range(n_epochs):
    ep_loss = 0
    for inp, tar in zip(X,y):
        inp = torch.autograd.Variable(
            torch.tensor( inp, dtype=torch.float )
        )
        tar = torch.autograd.Variable(
            torch.tensor( tar, dtype=torch.float )
        )
        inp = torch.reshape(inp,(3,1,1))
        tar = torch.reshape(tar,(1,1,1))
        
        optimizer.zero_grad()
        out = model(inp)
#         print(out[-1:].shape)
        loss = loss_fn(out[-1:],tar)
        ep_loss += loss
        loss.backward()
        optimizer.step()
    print(str(ep)+"  "+str(ep_loss / len(X))+"\r",end='',flush=True)

        

KeyboardInterrupt: 

In [92]:
val_loss = 0
for inp,tar in zip(data.X_test, data.y_test):
    inp = torch.autograd.Variable(
            torch.tensor( inp, dtype=torch.float )
        )
    tar = torch.autograd.Variable(
        torch.tensor( tar, dtype=torch.float )
    )
    inp = torch.reshape(inp,(3,1,1))
    tar = torch.reshape(tar,(1,1,1))
    out = model(inp)
    loss = loss_fn(out[-1:],tar)
    val_loss += loss
print(val_loss / len(data.X_test))

tensor(1.2307, grad_fn=<DivBackward0>)


In [55]:
# Here we define our model as a class
class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        lstm_out, self.hidden = self.lstm(input.view((len(input), self.batch_size, -1)))
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        return y_pred.view(-1)

lstm_input_size = 1
h1 = 1

model = LSTM(lstm_input_size, h1, batch_size=2)

In [56]:
num_epochs = 10
X_train = data.X_train
y_train = data.y_train
# -----------------------------------------------------
loss_fn = torch.nn.MSELoss(size_average=False)

optimiser = torch.optim.Adam(model.parameters())

#####################
# Train model
#####################

hist = np.zeros(num_epochs)

for t in range(num_epochs):
    # Clear stored gradient
    model.zero_grad()
    
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    model.hidden = model.init_hidden()
    
    # Forward pass
    y_pred = model(X_train)

    loss = loss_fn(y_pred, y_train)
    if t % 100 == 0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()

TypeError: data type not understood

In [54]:
X_train

array([[ 0.4781265 , -0.64181071,  0.93196523],
       [-0.64181071,  0.93196523,  1.24348954],
       [ 0.93196523,  1.24348954, -0.45840917],
       [ 1.24348954, -0.45840917,  0.30812605],
       [-0.45840917,  0.30812605,  0.35834108],
       [ 0.30812605,  0.35834108, -1.38662318],
       [ 0.35834108, -1.38662318,  0.69901411],
       [-1.38662318,  0.69901411,  0.24848427],
       [ 0.69901411,  0.24848427,  0.85752616],
       [ 0.24848427,  0.85752616, -0.11575259],
       [ 0.85752616, -0.11575259,  0.35100975],
       [-0.11575259,  0.35100975,  1.65124456],
       [ 0.35100975,  1.65124456, -1.3922242 ],
       [ 1.65124456, -1.3922242 ,  0.15814128],
       [-1.3922242 ,  0.15814128, -0.08669572],
       [ 0.15814128, -0.08669572, -0.18342843],
       [-0.08669572, -0.18342843, -1.28251734],
       [-0.18342843, -1.28251734,  0.19538482],
       [-1.28251734,  0.19538482,  0.99293658],
       [ 0.19538482,  0.99293658,  1.7376166 ],
       [ 0.99293658,  1.7376166 , -0.402

###########################################################################

In [36]:
hid_base = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))
inp_base = [torch.randn(1, 3) for _ in range(4)]

In [37]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
# inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5
inputs = inp_base

# initialize the hidden state.
# hidden = (torch.randn(1, 1, 3),
#           torch.randn(1, 1, 3))
hidden = hid_base

# for i in range(8):
#     # Step through the sequence one element at a time.
#     # after each step, hidden contains the hidden state.
#     out, hidden = lstm(inputs[i].view(1, 1, -1), hidden)
#     inputs.append(out)
#     print(out)
#     print(hidden)
#     print("-------------------")

# print("==============")

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
# hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
hidden = hid_base
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.1053,  0.4817, -0.1503]],

        [[-0.2229,  0.5812, -0.3411]],

        [[ 0.4357,  0.1472, -0.5547]],

        [[ 0.3102, -0.0966, -0.4476]]], grad_fn=<StackBackward>)
(tensor([[[ 0.3102, -0.0966, -0.4476]]], grad_fn=<StackBackward>), tensor([[[ 0.5090, -0.2803, -0.6513]]], grad_fn=<StackBackward>))


In [21]:
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5
inputs

[tensor([[0.6518, 0.7962, 0.3739]]),
 tensor([[-0.3925, -0.4316,  0.3713]]),
 tensor([[ 0.7826,  0.2304, -0.2615]]),
 tensor([[-0.3522, -0.6057,  0.8928]]),
 tensor([[-0.0165,  1.3197, -0.0099]])]

In [38]:
inputs.shape

torch.Size([4, 1, 3])